In [ ]:
%matplotlib inline
%load_ext Cython

from gwpy.timeseries import TimeSeries
import numpy as np
from astropy.coordinates import get_sun
from scipy.signal import butter, filtfilt, iirdesign, zpk2tf, freqz
from scipy.misc import logsumexp
from scipy.interpolate import RectBivariateSpline
import lal
import astropy.time as Time

from matplotlib import pyplot as pl

In [ ]:
# filtering functions
def iir_bandstops(fstops, fs, order=4):
    nyq = 0.5 * fs
    # Zeros zd, poles pd, and gain kd for the digital filter
    zd = np.array([])
    pd = np.array([])
    kd = 1

    # Notches
    for fstopData in fstops:
        fstop = fstopData[0]
        df = fstopData[1]
        df2 = fstopData[2]
        low = (fstop - df) / nyq
        high = (fstop + df) / nyq
        low2 = (fstop - df2) / nyq
        high2 = (fstop + df2) / nyq
        z, p, k = iirdesign([low,high], [low2,high2], gpass=1, gstop=6, ftype='ellip', output='zpk')
        zd = np.append(zd,z)
        pd = np.append(pd,p)

    # Set gain to one at 100 Hz...better not notch there
    bPrelim,aPrelim = zpk2tf(zd, pd, 1)
    outFreq, outg0 = freqz(bPrelim, aPrelim, 100/nyq)

    # Return the numerator and denominator of the digital filter
    b,a = zpk2tf(zd,pd,k)
    return b, a

def get_filter_coefs(det,fs=4096):
    # assemble the filter b,a coefficients:
    coefs = []

    # bandpass filter parameters
    lowcut = 100
    highcut = 150
    order = 4

    # bandpass filter coefficients
    nyq = 0.5*fs
    low = lowcut / nyq
    high = highcut / nyq
    bb, ab = butter(order, [low, high], btype='band')
    coefs.append((bb,ab))

    # Frequencies of notches at known instrumental spectral line frequencies.
    # You can see these lines in the ASD above, so it is straightforward to make this list.
    if det=='L1':
        notchesAbsolute = np.array([120.0, 139.94, 145.06, 108.992])
    elif det=='H1':
        notchesAbsolute = np.array([120.0, 139.95, 140.41, 108.992])
    else:
        print 'Error: Detector can only be H1 or L1'
    # notch filter coefficients:
    for notchf in notchesAbsolute:
        bn, an = iir_bandstops(np.array([[notchf,1,0.1]]), fs, order=4)
        coefs.append((bn,an))

    return coefs

# and then define the filter function:
def filter_data(data_in,coefs):
    data = data_in.copy()
    for coef in coefs:
        b,a = coef
        # filtfilt applies a linear filter twice, once forward and once backwards.
        # The combined filter has linear phase.
        data = filtfilt(b, a, data)
    return data

In [ ]:
# antenna response function
def antenna_response( gpsTime, ra, dec, psi, det ):
    # create detector-name map
    detMap = {'H1': lal.LALDetectorIndexLHODIFF,
              'L1': lal.LALDetectorIndexLLODIFF}

    try:
        detector=detMap[det]
    except KeyError:
        raise ValueError, "ERROR. Key %s is not a valid detector name." % (det)

    # get detector
    detval = lal.CachedDetectors[detector]
    response = detval.response

    # check if gpsTime is just a float or int, and if so convert into an array
    if isinstance(gpsTime, float) or isinstance(gpsTime, int):
        gpsTime = np.array([gpsTime])
    else: # make sure it's a numpy array
        gpsTime = np.copy(gpsTime)

    # if gpsTime is a list of regularly spaced values then use ComputeDetAMResponseSeries
    if len(gpsTime) == 1 or np.unique(np.diff(gpsTime)).size == 1:
        gpsStart = lal.LIGOTimeGPS( gpsTime[0] )
        dt = 0.
        if len(gpsTime) > 1:
            dt = gpsTime[1]-gpsTime[0]
        fp, fc = lal.ComputeDetAMResponseSeries(response, ra, dec, psi, gpsStart, dt, len(gpsTime))

        # return elements from Time Series
        return fp.data.data, fc.data.data
    else: # we'll have to work out the value at each point in the time series
        fp = np.zeros(len(gpsTime))
        fc = np.zeros(len(gpsTime))

        for i in range(len(gpsTime)):
            gps = lal.LIGOTimeGPS( gpsTime[i] )
            gmst_rad = lal.GreenwichMeanSiderealTime(gps)
            
            # actual computation of antenna factors
            fp[i], fc[i] = lal.ComputeDetAMResponse(response, ra, dec, psi, gmst_rad)

        return fp, fc

In [ ]:
starttime = 931076896
duration = 10
endtime = starttime+duration
dt = 1./16384.

# read in H1 data
H1cache = 'H1/H1cache.lcf'
strainH1 = TimeSeries.read(H1cache, channel='H1:LDAS-STRAIN', start=starttime, end=endtime)

# read in L1 data
L1cache = 'L1/L1cache.lcf'
strainL1 = TimeSeries.read(L1cache, channel='L1:LDAS-STRAIN', start=starttime, end=endtime)

# GPS times of data
gpstimes = np.arange(starttime, endtime, dt)

In [ ]:
# filter the data
coefsH  = get_filter_coefs('H1')
coefsL  = get_filter_coefs('L1')

filtstrainH1 = filter_data(strainH1, coefsH)
filtstrainL1 = filter_data(strainL1, coefsH)

In [ ]:
# remove first half of data (to remove filter effects)
filtstrainH1 = filtstrainH1[int(duration/(2*dt)):]
filtstrainL1 = filtstrainL1[int(duration/(2*dt)):]
gpstimes = gpstimes[int(duration/(2*dt)):]

In [ ]:
# downsample (via averaging)
dsfact = 8
dsstrainH1 = np.zeros(len(filtstrainH1)/dsfact)
dsstrainL1 = np.zeros(len(filtstrainL1)/dsfact)
for i in range(len(dsstrainH1)):
    j = dsfact*i + (dsfact/2)
    dsstrainH1[i] = np.mean(filtstrainH1[j-(dsfact/2):j+(dsfact/2)])
    dsstrainL1[i] = np.mean(filtstrainL1[j-(dsfact/2):j+(dsfact/2)])

# get central times of downsampled data
gpstimes = gpstimes[dsfact/2::dsfact]

In [ ]:
# get time delay between detectors for central time of data
gpsmiddle = gpstimes[int(len(gpstimes)/2)]
coords = get_sun(Time.Time(gpsmiddle, format='gps'))
ra = coords.ra.hour*np.pi/12. # right ascension in radians
dec = coords.dec.hour*np.pi/12. # declination in radians

detH1 = lal.CachedDetectors[lal.LALDetectorIndexLHODIFF]
detL1 = lal.CachedDetectors[lal.LALDetectorIndexLLODIFF]
tgps = lal.LIGOTimeGPS(int(gpsmiddle), int(1e9*(gpsmiddle-int(gpsmiddle))))

tdelay = lal.ArrivalTimeDiff(detH1.location, detL1.location, ra, dec, tgps)

dtnew = dt*dsfact
idxshift = np.rint(tdelay/dtnew)

In [ ]:
# range of polarisation angles (psi)
npsi = 35 # number of grid points in psi
psis = np.linspace(0., np.pi, npsi)

# get detector antenna patterns (just get at the start and end of the data and interpolate between these)
#fpH1 = np.zeros((npsi, 2))
#fcH1 = np.zeros((npsi, 2))
#fpL1 = np.zeros((npsi, 2))
#fcL1 = np.zeros((npsi, 2))
# atimes = np.array([gpstimes[0], gpstimes[-1]])
atimes = gpsmiddle
fpH1 = np.zeros(npsi)
fcH1 = np.zeros(npsi)
fpL1 = np.zeros(npsi)
fcL1 = np.zeros(npsi)

for i, psi in enumerate(psis):
    fpH1[i], fcH1[i] = antenna_response(atimes, ra, dec, psi, 'H1')
    fpL1[i], fcL1[i] = antenna_response(atimes, ra, dec, psi, 'L1')

# Bilinear interpolator (probably overkill!) for antenna patterns
#fpH1int = RectBivariateSpline(psis, atimes, fpH1, kx=1, ky=1)
#fcH1int = RectBivariateSpline(psis, atimes, fcH1, kx=1, ky=1)
#fpL1int = RectBivariateSpline(psis, atimes, fpL1, kx=1, ky=1)
#fcL1int = RectBivariateSpline(psis, atimes, fcL1, kx=1, ky=1)

In [ ]:
#gpstimesH1 = np.copy(gpstimes)
#gpstimesL1 = np.copy(gpstimes)

# shift data array so they line up
if idxshift >= 0: # shift the L1 data with respect to the H1 data
    dsstrainL1 = dstrainL1[idxshift:]
    dsstrainH1 = dstrainH1[0:len(dsstrainL1)] # shorten the H1 array to match
    #gpstimesL1 = gpstimesL1[idxshift:] # shifting times to correctly calculate antenna patterns is probably overkill!
    #gpstimesH1 = gpstimesH1[0:len(gpstimesL1)]
else: # shift the H1 data with respect to the L1 data
    dsstrainH1 = dsstrainH1[int(np.abs(idxshift)):]
    dsstrainL1 = dsstrainL1[0:len(dsstrainH1)] # shorten the L1 array to match
    #gpstimesH1 = gpstimesH1[int(np.abs(idxshift)):]
    #gpstimesL1 = gpstimesL1[0:len(gpstimesH1)]

In [ ]:
# set up the estimation of h0
# get the standard deviations of the data
sigmaH1 = np.std(dsstrainH1)
sigmaL1 = np.std(dsstrainL1)

numpoints = len(dsstrainH1)

# get the data inverse covariance matrix
invC = np.array([[1./sigmaH1**2, 0.], [0., 1./sigmaL1**2]])
detC = sigmaH1**2 * sigmaL1**2 # determinant of the covariance matrix

X = 5.
h0range = np.linspace(0., X*np.sqrt(2.*np.max([sigmaH1, sigmaL1])**2/numpoints), 25)
print h0range[-1]

# set up marginalised amplitudes covariances
sigmaAp = sigmaAc = 1. # THIS VALUE HAS A LARGE EFFECT ON THE RESULTS!
invSigma0 = np.array([[1./sigmaAp**2, 0.], [0., 1./sigmaAc**2]])
detSigma0 = sigmaAp**2 * sigmaAc**2

In [ ]:
# ORIGINAL VERSION IN WHICH THE ANSWER IS TOTALLY DEPENDENT ON THE sigmaAp AND sigmaAc VALUES

# get h0 posterior
logp = np.zeros(len(h0range))
logppsi = np.zeros(len(psis))

# normalisation
norm = 0.5*np.log(16.*np.pi**4*detSigma0*detC)
logdpsi_2 = np.log(0.5*(psis[1]-psis[0]))

for i in range(numpoints):
#for i in range(1):
    print i
    
    # data vector
    d = np.array([dsstrainH1[i], dsstrainL1[i]])
    d.shape = (2,1)
    
    invCd = np.dot(invC, d)
    dinvCd = np.vdot(d.T, invCd)
    
    # loop over h0 values
    for j, h0 in enumerate(h0range):
        # loop over psi value
        for k, psi in enumerate(psis):
            # get model design matrix
            #M = h0*np.array([[fpH1int(psi, gpstimesH1[i])[0,0], fpL1int(psi, gpstimesH1[i])[0,0]], [fcH1int(psi, gpstimesH1[i])[0,0], fcL1int(psi, gpstimesH1[i])[0,0]]])
            M = h0*np.array([[fpH1[k], fpL1[k]], [fcH1[k], fcL1[k]]])

            invSigma = np.dot(M.T, np.dot(invC, M)) + invSigma0
            Sigma = np.linalg.inv(invSigma)
            detSigma = np.linalg.det(Sigma)
            
            chi = np.dot(Sigma, np.dot(M.T, invCd))
            
            logppsi[k] = 0.5*np.log(detSigma) - norm - 0.5*(dinvCd + np.vdot(chi.T, np.dot(invSigma, chi)))
        
        # marginalise over psi for this h0
        logp[j] += logdpsi_2 + logsumexp([logsumexp(logppsi[:-1]), logsumexp(logppsi[1:])])

In [ ]:
%%cython

# SAME AS ABOVE BUT IN CYTHON

import numpy as np
cimport numpy as np

import scipy

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t
cdef double LOG2 = 0.6931471805599453094172321214581766

cdef extern from "math.h":
    double log(double x)
    double exp(double x)

cpdef logplus(double x, double y):
    """
    Calculate :math:`\log{(e^x + e^y)}` in a way that preserves numerical precision.
    """
    if (x > y):
        z = x + log(1 + exp(y - x))
    elif (x <= y):
        z = y + log(1 + exp(x - y))
    return z

cpdef double logtrapz(np.ndarray[DTYPE_t, ndim=1] lx, double t):
    """
    This function calculates the integral via the trapezium rule of the logarithm of a function `lx`.
    """
    assert lx.dtype == DTYPE
    ldts = log(t) - LOG2
    cdef double B = (-1)*np.inf

    cdef int i = 0
    cdef double z
    cdef int loopmax = len(lx)-1

    for i in range(loopmax):
        z = logplus(lx[i], lx[i+1])
        z = z + ldts
        B = logplus(z,B)
    return B

# instead search over the standard deviation and mean of the underlying amplitude distribution
cpdef calc_log_like(np.ndarray[DTYPE_t, ndim=1] dataH1, np.ndarray[DTYPE_t, ndim=1] dataL1,
                    np.ndarray[DTYPE_t, ndim=1] fpH1, np.ndarray[DTYPE_t, ndim=1] fcH1,
                    np.ndarray[DTYPE_t, ndim=1] fpL1, np.ndarray[DTYPE_t, ndim=1] fcL1,
                    np.ndarray[DTYPE_t, ndim=1] psis, np.ndarray[DTYPE_t, ndim=1] muArange,
                    np.ndarray[DTYPE_t, ndim=1] sigmaArange):
    cdef int numpoints = len(dataH1)

    cdef double sigmaH1 = np.std(dataH1)
    cdef double sigmaL1 = np.std(dataL1)

    logppsi = np.zeros(len(psis))
    logpp = np.ones((len(muArange), len(sigmaArange)))*-np.inf

    cdef double dpsi = psis[1]-psis[0] 

    invC = np.array([[1./sigmaH1**2, 0.], [0., 1./sigmaL1**2]])
    detC = sigmaH1**2 * sigmaL1**2 # determinant of the covariance matrix

    #for i in range(numpoints):
    for i in range(10):
        # data vector
        d = np.array([dataH1[i], dataL1[i]])
        d.shape = (2,1)

        invCd = np.dot(invC, d)
        dinvCd = np.vdot(d.T, invCd)
    
        # get model components
        MinvCd = np.zeros((len(psis), 2, 1))
        MinvCM = np.zeros((len(psis), 2, 2))
        for n, psi in enumerate(psis):
            M = np.array([[fpH1[n], fpL1[n]], [fcH1[n], fcL1[n]]])
            MinvCM[n] = np.dot(M.T, np.dot(invC, M))
            MinvCd[n] = np.dot(M.T, invCd)

        for j, muA in enumerate(muArange):
            xi0 = np.array([muA, muA])
            xi0.shape = (2,1)
            for k, sigmaA in enumerate(sigmaArange):
                invSigma0 = np.array([[1./sigmaA**2, 0.], [0., 1./sigmaA**2]])
                detSigma0 = sigmaA**4
    
                invSigma0xi = np.dot(invSigma0, xi0)
                xixi = np.vdot(xi0.T, np.dot(invSigma0, xi0))

                # normalisation
                norm = 0.5*np.log(16.*np.pi**4*detSigma0*detC)

                for n, psi in enumerate(psis):
                    # get model design matrix
                    invSigma = MinvCM[n] + invSigma0
                    Sigma = np.linalg.inv(invSigma)
                    detSigma = np.linalg.det(Sigma)

                    chi = np.dot(Sigma, MinvCd[n] + invSigma0xi)

                    logppsi[n] = 0.5*np.log(detSigma) - norm - 0.5*(dinvCd + np.vdot(chi.T, np.dot(invSigma, chi)) + xixi)

                logpp[j,k] = logplus(logpp[j,k], logtrapz(logppsi, dpsi))
                
    return logpp
    

In [ ]:
X = 50.
Amax = X*np.sqrt(2.*np.max([sigmaH1, sigmaL1])**2/numpoints)
sigmaArange = np.linspace(Amax/1.e6, Amax, 20)
muArange = np.linspace(0., Amax, 25)

logpp = calc_log_like(dsstrainH1, dsstrainL1, fpH1, fcH1, fpL1, fcL1, psis, muArange, sigmaArange)

In [ ]:
pl.imshow(np.fliplr(np.flipud(np.copy(logpp.T))), extent=(muArange[0], muArange[-1], sigmaArange[0], sigmaArange[-1]), aspect='auto')

In [ ]:
# marginalise over sigmaA
dsigma_2 = np.log(0.5*(sigmaArange[1]-sigmaArange[0]))
margpost = [dsigma_2 + logsumexp([logsumexp(row[:-1]), logsumexp(row[1:])]) for row in logpp]
margpost2 = [dsigma_2 + logsumexp([logsumexp(row[:-1]), logsumexp(row[1:])]) for row in logpp2]

In [ ]:
pl.plot(muArange, np.exp(margpost-np.max(margpost)), 'b', muArange, np.exp(margpost2-np.max(margpost2)), 'rx')

In [ ]:

pl.plot(muArange, np.exp(margpost-np.max(margpost)), 'r')

In [ ]:
%%cython

# function to marginalise over flat priors on unknown amplitudes

import numpy as np
cimport numpy as np

import scipy

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t
cdef double LOG2 = 0.6931471805599453094172321214581766

cdef extern from "math.h":
    double log(double x)
    double exp(double x)

cpdef logplus(double x, double y):
    """
    Calculate :math:`\log{(e^x + e^y)}` in a way that preserves numerical precision.
    """
    if (x > y):
        z = x + log(1 + exp(y - x))
    elif (x <= y):
        z = y + log(1 + exp(x - y))
    return z

cpdef double logtrapz(np.ndarray[DTYPE_t, ndim=1] lx, double t):
    """
    This function calculates the integral via the trapezium rule of the logarithm of a function `lx`.
    """
    assert lx.dtype == DTYPE
    ldts = log(t) - LOG2
    cdef double B = (-1)*np.inf

    cdef int i = 0
    cdef double z
    cdef int loopmax = len(lx)-1

    for i in range(loopmax):
        z = logplus(lx[i], lx[i+1])
        z = z + ldts
        B = logplus(z,B)
    return B

# instead search over the standard deviation and mean of the underlying amplitude distribution
cpdef calc_log_like_h(np.ndarray[DTYPE_t, ndim=1] dataH1, np.ndarray[DTYPE_t, ndim=1] dataL1,
                    np.ndarray[DTYPE_t, ndim=1] fpH1, np.ndarray[DTYPE_t, ndim=1] fcH1,
                    np.ndarray[DTYPE_t, ndim=1] fpL1, np.ndarray[DTYPE_t, ndim=1] fcL1,
                    np.ndarray[DTYPE_t, ndim=1] psis, np.ndarray[DTYPE_t, ndim=1] h0s,
                    np.ndarray[DTYPE_t, ndim=1] muhrange, np.ndarray[DTYPE_t, ndim=1] sigmahrange):
    cdef int numpoints = len(dataH1)

    cdef double sigmaH1 = np.std(dataH1)
    cdef double sigmaL1 = np.std(dataL1)

    logppsi = np.zeros(len(psis))
    
    logppsih = np.zeros((len(psis), len(h0s)))
    logpp = np.zeros((len(muhrange), len(sigmahrange)))

    cdef double dpsi = psis[1]-psis[0]
    cdef double dh0 = h0s[1]-h0s[0]

    invC = np.array([[1./sigmaH1**2, 0.], [0., 1./sigmaL1**2]])
    detC = sigmaH1**2 * sigmaL1**2 # determinant of the covariance matrix
 
    print invC

    # get model components
    Cprime = np.zeros((len(psis), len(h0s), 2, 2))
    MinvCM = np.zeros((len(psis), len(h0s), 2, 2))
    lognorm = np.zeros((len(psis), len(h0s)))
    for i, psi in enumerate(psis):
        M = np.array([[fpH1[i], fpL1[i]], [fcH1[i], fcL1[i]]])
        for j, h0 in enumerate(h0s):
            MM = h0*M
            MinvCM[i,j] = np.dot(MM.T, np.dot(invC, MM))
            invMinvCM = np.linalg.inv(MinvCM[i,j])
            lognorm[i,j] = 0.5*(log(np.linalg.det(invMinvCM)) - log(detC))
            print lognorm[i,j]
            Cprime[i,j] = invC - np.dot(np.dot(invC, MM), np.dot(invMinvCM, np.dot(MM.T, invC)))
            print np.dot(np.dot(invC, MM), np.dot(invMinvCM, np.dot(MM.T, invC)))
            
    logh0prob = np.zeros((len(muhrange), len(sigmahrange), len(h0s)))
    for i, muh in enumerate(muhrange):
        for j, sigmah in enumerate(sigmahrange):
            for k, h0 in enumerate(h0s):
                logh0prob[i,j,k] = -log(sigmah)-0.5*log(2.*np.pi)-0.5*((h0-muh)**2/sigmah**2)

    #for i in range(numpoints):
    for i in range(1):
        # data vector
        d = np.array([dataH1[i], dataL1[i]])
        d.shape = (2,1)

        for i in range(len(psis)):
            for j in range(len(h0s)):
                logppsih[i,j] = lognorm[i,j] - 0.5*(np.vdot(d.T, np.dot(Cprime[i,j], d)))
        
        for i in range(len(muhrange)):
            for j in range(len(sigmahrange)):
                # marginalise over h0 and psi
                for k, row in enumerate(logppsih):
                    logppsi[k] = logtrapz(row+logh0prob[i,j], dh0)
                logpp[i,j] += logtrapz(logppsi, dpsi)

    return logppsih


In [ ]:
X = 500.
hmax = X*np.sqrt(2.*np.max([sigmaH1, sigmaL1])**2/numpoints)
sigmahrange = np.linspace(hmax/1.e6, hmax, 20)
muhrange = np.linspace(0., hmax, 25)
h0s = np.linspace(hmax/10., hmax, 2)

logpp = calc_log_like_h(dsstrainH1, dsstrainL1, fpH1, fcH1, fpL1, fcL1, psis, h0s, muhrange, sigmahrange)

In [ ]:
X = 500.
hmax = X*np.sqrt(2.*np.max([sigmaH1, sigmaL1])**2/numpoints)
sigmahrange = np.linspace(hmax/20, 20.*hmax, 20)
muhrange = np.linspace(0., 20.*hmax, 25)
h0s = np.linspace(hmax/100., 20.*hmax, 20)
Acrange = np.linspace(0.0, 1., 10)

logpp = calc_log_like_new(dsstrainH1[0:], dsstrainL1[0:], fpH1, fcH1, fpL1, fcL1, psis, Acrange, h0s, muhrange, sigmahrange)

pl.plot(h0s, np.exp(logpp-np.max(logpp)))

# marginalise over sigma
#logmuprob = np.zeros(len(muhrange))
#dsigma = sigmahrange[1]-sigmahrange[0]
#for i in range(len(muhrange)):
#    logmuprob[i] = logtrapz(logpp[i], dsigma)

# marginalise over mu
#logsigmaprob = np.zeros(len(sigmahrange))
#dmu = muhrange[1]-muhrange[0]
#for i in range(len(sigmahrange)):
#    logsigmaprob[i] = logtrapz(logpp[:,i], dmu)

# plot mu distribution
#pl.plot(muhrange, np.exp(logmuprob-np.max(logmuprob)))

# plot sigma distribution
#pl.plot(sigmahrange, np.exp(logsigmaprob-np.max(logsigmaprob)))

In [ ]:
%%cython

# ESTIMATE THE MEAN VALUES OF EXPONENTIAL DISTRIBUTIONS ON APLUS AND ACROSS

import numpy as np
cimport numpy as np

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t
cdef double LOG2 = 0.6931471805599453094172321214581766

cdef extern from "math.h":
    double log(double x)
    double exp(double x)
    double sqrt(double x)
    double fabs(double x)
    
cpdef logplus(double x, double y):
    """
    Calculate :math:`\log{(e^x + e^y)}` in a way that preserves numerical precision.
    """
    if (x > y):
        z = x + log(1 + exp(y - x))
    elif (x <= y):
        z = y + log(1 + exp(x - y))
    return z

cpdef double logtrapz(np.ndarray[DTYPE_t, ndim=1] lx, double t):
    """
    This function calculates the integral via the trapezium rule of the logarithm of a function `lx`.
    """
    assert lx.dtype == DTYPE
    ldts = log(t) - LOG2
    cdef double B = (-1)*np.inf

    cdef int i = 0
    cdef double z
    cdef int loopmax = len(lx)-1

    for i in range(loopmax):
        z = logplus(lx[i], lx[i+1])
        z = z + ldts
        B = logplus(z,B)
    return B
    
# code up a function for which one of the amplitudes is anlytically marginalised over between -1 and 1
cpdef calc_log_like_new2(np.ndarray[DTYPE_t, ndim=1] dataH1, np.ndarray[DTYPE_t, ndim=1] dataL1,
                         np.ndarray[DTYPE_t, ndim=1] fpH1, np.ndarray[DTYPE_t, ndim=1] fcH1,
                         np.ndarray[DTYPE_t, ndim=1] fpL1, np.ndarray[DTYPE_t, ndim=1] fcL1,
                         np.ndarray[DTYPE_t, ndim=1] psis, np.ndarray[DTYPE_t, ndim=1] Acrange,
                         np.ndarray[DTYPE_t, ndim=1] Aprange, np.ndarray[DTYPE_t, ndim=1] muAcrange,
                         np.ndarray[DTYPE_t, ndim=1] muAprange):
    cdef int numpoints = len(dataH1)

    cdef double sigmaH1 = np.std(dataH1)
    cdef double sigmaL1 = np.std(dataL1)
    cdef double varH1 = sigmaH1**2
    cdef double varL1 = sigmaL1**2

    cdef double logpriorpsi = -log(psis[-1]-psis[0])

    cdef double dpsi = psis[1]-psis[0]
    cdef double dAc = Acrange[1]-Acrange[0]
    cdef double dAp = Aprange[1]-Aprange[0]

    logppsi = np.zeros(len(psis))
    logpA = np.zeros((len(Aprange), len(Acrange)))
    logpp = np.zeros((len(muAprange), len(muAcrange)))

    logAcprob = np.zeros((len(muAcrange), len(Acrange)))
    absAc = np.abs(Acrange)
    for i, muAc in enumerate(muAcrange):
        logAcprob[i] = -log(muAc) - absAc/muAc

    logApprob = np.zeros((len(muAprange), len(Aprange)))
    absAp = np.abs(Aprange)
    for i, muAp in enumerate(muAprange):
        logApprob[i] = -log(muAp) - absAp/muAp
 
    logA = np.zeros(len(Aprange))

    cdef double lognorm = log(2.*np.pi*sigmaH1*sigmaL1)
    
    afH1 = np.zeros((len(Aprange), len(Acrange), len(psis)))
    afL1 = np.zeros((len(Aprange), len(Acrange), len(psis)))
    for i in range(len(Aprange)):
        for j in range(len(Acrange)):
            for k in range(len(psis)):
                afH1[i,j,k] = (Aprange[i]*fpH1[k] + Acrange[j]*fcH1[k])
                afL1[i,j,k] = (Aprange[i]*fpL1[k] + Acrange[j]*fcL1[k])
    
    #for i in range(numpoints):
    for n in range(100):
        for i in range(len(Aprange)):
            for j in range(len(Acrange)):
                for k in range(len(psis)):
                    logppsi[k] = -0.5*(dataH1[n] - afH1[i,j,k])**2/varH1
                    logppsi[k] -= 0.5*(dataL1[n] - afL1[i,j,k])**2/varL1

                # marginalise over psi
                logpA[i,j] = logtrapz(logppsi, dpsi)
                #logpA[i,j] += logpriorpsi

        #logph -= lognorm

        # get posterior on muAp and muAc parameters (means of exponential distributions)
        for i in range(len(muAprange)):
            for j in range(len(muAcrange)):
                for k in range(len(Aprange)):
                    logA[k] = logtrapz(logpA[k]+logAcprob[j], dAc)
                logpp[i,j] += logtrapz(logA+logApprob[i], dAp)
                print logpp[i,j]

    #return logpp
    return logpA

In [ ]:
X = 500.
hmax = X*np.sqrt(2.*np.max([sigmaH1, sigmaL1])**2/numpoints)
muAcrange = np.linspace(hmax/20., 2.*hmax, 50)
muAprange = np.linspace(hmax/20., 2.*hmax, 50)
Acrange = np.linspace(-2.*hmax, 2.*hmax, 50)
Aprange = np.linspace(-2.*hmax, 2.*hmax, 50)

logpp = calc_log_like_new2(dsstrainH1[0:], dsstrainL1[0:], fpH1, fcH1, fpL1, fcL1, psis, Aprange, Acrange, muAcrange, muAprange)

#pl.plot(Aprange, np.exp(logpp-np.max(logpp)))
#pl.imshow(logpp)

#dAc = muAcrange[1]-muAcrange[0]
#logpAp = [logtrapz(row, dAc) for row in logpp]
#pAp = np.exp(logpAp-np.max(logpAp))

#dAp = muAprange[1]-muAprange[0]
#logpAc = [logtrapz(row, dAp) for row in logpp.T]
#pAc = np.exp(logpAc-np.max(logpAc))
#pl.plot(muAcrange, pAc, 'r')

In [ ]:
X = 500.
hmax = X*np.sqrt(2.*np.max([sigmaH1, sigmaL1])**2/numpoints)
muAcrange = np.linspace(hmax/20., 2.*hmax, 50)
muAprange = np.linspace(hmax/20., 2.*hmax, 50)
Amin = -2.*hmax
Amax = 2.*hmax

logpp = calc_log_like(dsstrainH1, dsstrainL1, fpH1, fcH1, fpL1, fcL1, psis, Amin, Amax, muAcrange, muAprange)

In [ ]:
# try using scipy's numerical integration routines (TOO SLOW)
from scipy.integrate import tplquad

# define the likelihood function
def likelihood(Ac, Ap, psi, dH, dL, sigmaH, sigmaL, fpH, fcH, fpL, fcL, psis, muAp, muAc):
    fpnH = np.interp(psi, psis, fpH)
    fcnH = np.interp(psi, psis, fcH)
    fpnL = np.interp(psi, psis, fpL)
    fcnL = np.interp(psi, psis, fcL)
    afH = (Ap*fpnH + Ac*fcnH)
    afL = (Ap*fpnL + Ac*fcnL)
    varH = sigmaH**2
    varL = sigmaL**2
    absAc = np.abs(Ac)
    absAp = np.abs(Ap)

    return np.exp(-0.5*(((dH - afH)**2/varH) + ((dL - afL)**2/varL))-(absAp/muAp)-(absAc/muAc))/(muAc*muAp)

def rAmin(x, Amin):
    return Amin

def rAmax(x, Amax):
    return Amax

def rrAmin(x, y, Amin):
    return Amin

def rrAmax(x, y, Amax):
    return Amax

def calc_log_like(dataH1, dataL1, fpH1, fcH1, fpL1, fcL1, psis, Amin, Amax, muAcrange, muAprange):
    numpoints = len(dataH1)

    sigmaH1 = np.std(dataH1)
    sigmaL1 = np.std(dataL1)
    logpp = np.zeros((len(muAprange), len(muAcrange)))

    #for i in range(numpoints):
    for n in range(1):
        # get posterior on muAp and muAc parameters (means of exponential distributions)
        for i in range(len(muAprange)):
            for j in range(len(muAcrange)):
                I = tplquad(likelihood, 0., np.pi, lambda x: rAmin(x, Amin),
                            lambda x: rAmax(x, Amax), lambda x, y: rrAmin(x, y, Amin),
                            lambda x, y: rrAmax(x, y, Amax),
                            args=(dataH1[n], dataL1[n], sigmaH1, sigmaL1, fpH1,
                            fcH1, fpL1, fcL1, psis, muAprange[i], muAcrange[j]), epsrel=5e-2)
                print np.log(I[0])
                logpp[i,j] += np.log(I[0])
    
    return logpp

In [ ]:
X = 500.
hmax = X*np.sqrt(2.*np.max([sigmaH1, sigmaL1])**2/numpoints)
muAcrange = np.linspace(hmax/20., 2.*hmax, 50)
muAprange = np.linspace(hmax/20., 2.*hmax, 50)
Amin = -2.*hmax
Amax = 2.*hmax

logpp = calc_log_like(dsstrainH1, dsstrainL1, fpH1, fcH1, fpL1, fcL1, psis, Amin, Amax, muAcrange, muAprange)